## Load and format data

In [5]:
import pandas as pd
import numpy as np
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from transformers import pipeline




In [25]:
# Load VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Load the JSON file into a DataFrame
file_path = 'data/sarcasm_data.json'
df = pd.read_json(file_path).transpose()

# Reset the index to turn the first element into a new column
df = df.reset_index()

# Define a function to apply sentiment analysis to a text
def get_sentiment(text):
    return analyzer.polarity_scores(text)

# Apply sentiment analysis to the 'utterance' column
df['sentiment_utterance'] = df['utterance'].apply(get_sentiment)

# Apply sentiment analysis to the 'context' column
df['sentiment_context_all'] = df['context'].apply(get_sentiment)

# Apply sentiment analysis to each sentence in the 'context' column
df['sentiment_context_per_sentence'] = df['context'].apply(lambda context: [get_sentiment(sentence) for sentence in context])

# Visualize the DataFrame
df

,index,utterance,speaker,context,context_speakers,show,sarcasm,sentiment_utterance,sentiment_context_all,sentiment_context_per_sentence
0,160,It's just a privilege to watch your mind at work.,SHELDON,[I never would have identified the fingerprint...,"[LEONARD, SHELDON]",BBT,True,"{'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun..."
1,170,I don't think I'll be able to stop thinking ab...,PENNY,[This is one of my favorite places to kick bac...,"[HOWARD, PENNY, HOWARD, HOWARD, HOWARD, PENNY,...",BBT,True,"{'neg': 0.18, 'neu': 0.82, 'pos': 0.0, 'compou...","{'neg': 0.0, 'neu': 0.871, 'pos': 0.129, 'comp...","[{'neg': 0.0, 'neu': 0.705, 'pos': 0.295, 'com..."
2,180,"Since it's not bee season, you can have my epi...",SHELDON,"[Here we go. Pad thai, no peanuts., But does i...","[LEONARD, HOWARD, LEONARD]",BBT,False,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.143, 'neu': 0.857, 'pos': 0.0, 'comp...","[{'neg': 0.268, 'neu': 0.732, 'pos': 0.0, 'com..."
3,190,"Lois Lane is falling, accelerating at an initi...",SHELDON,[A marathon? How many Superman movies are ther...,"[PENNY, SHELDON, PENNY, SHELDON, SHELDON, PENN...",BBT,False,"{'neg': 0.058, 'neu': 0.851, 'pos': 0.091, 'co...","{'neg': 0.0, 'neu': 0.906, 'pos': 0.094, 'comp...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun..."
4,1105,I'm just inferring this is a couch because the...,SHELDON,"[Great Caesar's ghost, look at this place., So...","[SHELDON, LEONARD, SHELDON, SHELDON, SHELDON, ...",BBT,True,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","{'neg': 0.097, 'neu': 0.815, 'pos': 0.088, 'co...","[{'neg': 0.202, 'neu': 0.439, 'pos': 0.36, 'co..."
...,...,...,...,...,...,...,...,...,...,...
685,2169,"Hes not right for the part, and if I suggest h...",CHANDLER,"[What am I gonna do now?, Just pass the tape a...","[CHANDLER, RACHEL]",FRIENDS,True,"{'neg': 0.102, 'neu': 0.898, 'pos': 0.0, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun..."
686,2235,"Oh yeah he has a caretaker his older brother, ...",CHANDLER,"[Helo! Anybody in there order a celebrity?, Wh...","[JOEY, PERSON, CHANDLER, PERSON]",FRIENDS,False,"{'neg': 0.0, 'neu': 0.858, 'pos': 0.142, 'comp...","{'neg': 0.062, 'neu': 0.751, 'pos': 0.187, 'co...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun..."
687,234,Is it me or the greetings gone downhill around...,CHANDLER,"[Hey, You son of a bitch!]","[CHANDLER, JOEY]",FRIENDS,True,"{'neg': 0.0, 'neu': 0.763, 'pos': 0.237, 'comp...","{'neg': 0.506, 'neu': 0.494, 'pos': 0.0, 'comp...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun..."
688,2608,"You are right, by saying nice, I am virtually ...",CHANDLER,"[Did I go to this school?, Hey, there's Missy ...","[CHANDLER, ROSS, CHANDLER, ROSS]",FRIENDS,True,"{'neg': 0.0, 'neu': 0.781, 'pos': 0.219, 'comp...","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun..."


### Visualisation format SA context and utterance

In [46]:
df_context = df['sentiment_context_all']

print("context all sequences:", df['sentiment_context_all'][0])
df['sentiment_context_all'][0]
print("context per sequence:", df['sentiment_context_per_sentence'][0])
print("utterance:", df['sentiment_utterance'][0] )

context all sequences: {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
context per sequence: [{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}, {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
utterance: {'neg': 0.0, 'neu': 0.783, 'pos': 0.217, 'compound': 0.3612}


## Format Data for Training in format 1 sentiment for all context sentences

In [13]:
# Converts sarcasm True to 1 and False to 0
y = df['sarcasm'].astype(int).tolist()

context_all = df["sentiment_context_all"]
context_sentences = df["sentiment_context_per_sentence"]
utterance = df["sentiment_utterance"]

# Extracting 'neg', 'neu', and 'pos' per sentence and add index to keep context 
X = np.array([ [context_all[i]['neg'], context_all[i]['neu'], context_all[i]['pos'], utterance[i]['neg'], utterance[i]['neu'], utterance[i]['pos']] for i in range(context_all.shape[0]) ])

# Split the dataset into 80% training and 20% testing
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pd.DataFrame(X, columns=['context_neg', 'context_neu', 'context_pos', 'utterance_neg', 'utterance_neu', 'utterance_pos'])



,context_neg,context_neu,context_pos,utterance_neg,utterance_neu,utterance_pos
0,0.000,1.000,0.000,0.000,0.783,0.217
1,0.000,0.871,0.129,0.180,0.820,0.000
2,0.143,0.857,0.000,0.000,1.000,0.000
3,0.000,0.906,0.094,0.058,0.851,0.091
4,0.097,0.815,0.088,0.000,1.000,0.000
...,...,...,...,...,...,...
685,0.000,1.000,0.000,0.102,0.898,0.000
686,0.062,0.751,0.187,0.000,0.858,0.142
687,0.506,0.494,0.000,0.000,0.763,0.237
688,0.000,1.000,0.000,0.000,0.781,0.219


## Format DataFrame: 1 emotion per sentence + Utterance

In [48]:
# Define the keys for the floats
keys = ['neg', 'neu', 'pos']

# Initialize a list to hold the resulting arrays
X_sentences = []

# Process each array (list of dictionaries)
for idx, sentences in enumerate(context_sentences):
    # Ensure each list has at least 3 dictionaries, padding if necessary
    sentences.extend([{'neg': 0.0, 'neu': 0.0, 'pos': 0.0}] * (3 - len(sentences)))

    # Extract the relevant values from the last 3 context sentences and the corresponding utterance
    context_values = [value for sentence in sentences[-3:] for value in (sentence['neg'], sentence['neu'], sentence['pos'])]
    utterance_values = [utterance[idx][key] for key in keys]

    # Combine the context and utterance values
    X_sentences.append(context_values + utterance_values)

# Split the dataset into 80% training and 20% testing
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_sentences, y, test_size=0.2, random_state=42)

# Create a DataFrame with appropriate column names
columns = [f"Ctx_{i}_{key}" for i in range(3, 0, -1) for key in keys] + [f"Utr_{key}" for key in keys]
pd.DataFrame(X_sentences, columns=columns)



,Ctx_3_neg,Ctx_3_neu,Ctx_3_pos,Ctx_2_neg,Ctx_2_neu,Ctx_2_pos,Ctx_1_neg,Ctx_1_neu,Ctx_1_pos,Utr_neg,Utr_neu,Utr_pos
0,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.00,0.000,0.783,0.217
1,0.000,1.000,0.000,0.000,0.426,0.574,0.000,1.000,0.00,0.180,0.820,0.000
2,0.268,0.732,0.000,0.000,1.000,0.000,0.116,0.884,0.00,0.000,1.000,0.000
3,0.000,1.000,0.000,0.000,0.649,0.351,0.000,1.000,0.00,0.058,0.851,0.091
4,0.349,0.651,0.000,0.000,0.435,0.565,0.000,1.000,0.00,0.000,1.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.000,1.000,0.000,0.000,1.000,0.000,0.000,0.000,0.00,0.102,0.898,0.000
686,0.000,1.000,0.000,0.105,0.702,0.193,0.000,0.750,0.25,0.000,0.858,0.142
687,0.000,1.000,0.000,0.506,0.494,0.000,0.000,0.000,0.00,0.000,0.763,0.237
688,0.000,1.000,0.000,0.000,0.000,1.000,0.000,1.000,0.00,0.000,0.781,0.219


# Train and Test

## Overal context score

In [49]:
# Create a Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train, y_train)

# Predict on the test data
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

Accuracy: 0.49
Confusion Matrix:
[[42 11]
 [60 25]]
Classification Report:
              precision    recall  f1-score   support

           0       0.41      0.79      0.54        53
           1       0.69      0.29      0.41        85

    accuracy                           0.49       138
   macro avg       0.55      0.54      0.48       138
weighted avg       0.59      0.49      0.46       138



## Context score per sequence

In [50]:
# Create a Logistic Regression model
model = LogisticRegression()

# Train the model
model.fit(X_train_all, y_train_all)

# Predict on the test data
y_pred_all = model.predict(X_test_all)

# Evaluate the model
accuracy = accuracy_score(y_test_all, y_pred_all)
conf_matrix = confusion_matrix(y_test_all, y_pred_all)
report = classification_report(y_test_all, y_pred_all)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(report)

Accuracy: 0.49
Confusion Matrix:
[[31 22]
 [49 36]]
Classification Report:
              precision    recall  f1-score   support

           0       0.39      0.58      0.47        53
           1       0.62      0.42      0.50        85

    accuracy                           0.49       138
   macro avg       0.50      0.50      0.48       138
weighted avg       0.53      0.49      0.49       138



## Transformers 

In [7]:
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
def hartmann_sentiment(text):
    text = classifier(text)
    # Assuming 'text' is already defined and contains the necessary data
    sentiments = [text[0][i]['score'] for i in range(6)]
    sentiment = np.argmax(sentiments)
    return sentiment
text = "I love this!"
hartmann_sentiment(text)

c:\Users\camil\anaconda3\lib\site-packages\transformers\pipelines\text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


3

## Find sentiment to each 

In [8]:
# Load the JSON file into a DataFrame
file_path = 'data/sarcasm_data.json'
df_h = pd.read_json(file_path).transpose()


# Apply sentiment analysis to the 'utterance' column
df_h['sentiment_utterance'] = df_h['utterance'].apply(hartmann_sentiment)

# Apply sentiment analysis to the 'context' column
df_h['sentiment_context_all'] = df_h['context'].apply(hartmann_sentiment)


# Visualize the DataFrame
df_h

,utterance,speaker,context,context_speakers,show,sarcasm,sentiment_utterance,sentiment_context_all
160,It's just a privilege to watch your mind at work.,SHELDON,[I never would have identified the fingerprint...,"[LEONARD, SHELDON]",BBT,True,4,4
170,I don't think I'll be able to stop thinking ab...,PENNY,[This is one of my favorite places to kick bac...,"[HOWARD, PENNY, HOWARD, HOWARD, HOWARD, PENNY,...",BBT,True,4,4
180,"Since it's not bee season, you can have my epi...",SHELDON,"[Here we go. Pad thai, no peanuts., But does i...","[LEONARD, HOWARD, LEONARD]",BBT,False,4,4
190,"Lois Lane is falling, accelerating at an initi...",SHELDON,[A marathon? How many Superman movies are ther...,"[PENNY, SHELDON, PENNY, SHELDON, SHELDON, PENN...",BBT,False,4,4
1105,I'm just inferring this is a couch because the...,SHELDON,"[Great Caesar's ghost, look at this place., So...","[SHELDON, LEONARD, SHELDON, SHELDON, SHELDON, ...",BBT,True,4,2
...,...,...,...,...,...,...,...,...
2169,"Hes not right for the part, and if I suggest h...",CHANDLER,"[What am I gonna do now?, Just pass the tape a...","[CHANDLER, RACHEL]",FRIENDS,True,0,0
2235,"Oh yeah he has a caretaker his older brother, ...",CHANDLER,"[Helo! Anybody in there order a celebrity?, Wh...","[JOEY, PERSON, CHANDLER, PERSON]",FRIENDS,False,1,0
234,Is it me or the greetings gone downhill around...,CHANDLER,"[Hey, You son of a bitch!]","[CHANDLER, JOEY]",FRIENDS,True,4,4
2608,"You are right, by saying nice, I am virtually ...",CHANDLER,"[Did I go to this school?, Hey, there's Missy ...","[CHANDLER, ROSS, CHANDLER, ROSS]",FRIENDS,True,1,4


In [26]:
# Load the JSON file into a DataFrame
file_path = 'data/sarcasm_data.json'
df_h = pd.read_json(file_path).transpose()
df_h

,utterance,speaker,context,context_speakers,show,sarcasm
160,It's just a privilege to watch your mind at work.,SHELDON,[I never would have identified the fingerprint...,"[LEONARD, SHELDON]",BBT,True
170,I don't think I'll be able to stop thinking ab...,PENNY,[This is one of my favorite places to kick bac...,"[HOWARD, PENNY, HOWARD, HOWARD, HOWARD, PENNY,...",BBT,True
180,"Since it's not bee season, you can have my epi...",SHELDON,"[Here we go. Pad thai, no peanuts., But does i...","[LEONARD, HOWARD, LEONARD]",BBT,False
190,"Lois Lane is falling, accelerating at an initi...",SHELDON,[A marathon? How many Superman movies are ther...,"[PENNY, SHELDON, PENNY, SHELDON, SHELDON, PENN...",BBT,False
1105,I'm just inferring this is a couch because the...,SHELDON,"[Great Caesar's ghost, look at this place., So...","[SHELDON, LEONARD, SHELDON, SHELDON, SHELDON, ...",BBT,True
...,...,...,...,...,...,...
2169,"Hes not right for the part, and if I suggest h...",CHANDLER,"[What am I gonna do now?, Just pass the tape a...","[CHANDLER, RACHEL]",FRIENDS,True
2235,"Oh yeah he has a caretaker his older brother, ...",CHANDLER,"[Helo! Anybody in there order a celebrity?, Wh...","[JOEY, PERSON, CHANDLER, PERSON]",FRIENDS,False
234,Is it me or the greetings gone downhill around...,CHANDLER,"[Hey, You son of a bitch!]","[CHANDLER, JOEY]",FRIENDS,True
2608,"You are right, by saying nice, I am virtually ...",CHANDLER,"[Did I go to this school?, Hey, there's Missy ...","[CHANDLER, ROSS, CHANDLER, ROSS]",FRIENDS,True


## Find sentiment for 3 context sentences

In [28]:
# Apply sentiment analysis to each sentence in the 'context' column
df_h['sentiment_context_per_sentence'] = df_h['context'].apply(lambda context: [hartmann_sentiment(sentence) for sentence in context])

In [10]:
df_h

,utterance,speaker,context,context_speakers,show,sarcasm,sentiment_utterance,sentiment_context_all,sentiment_context_per_sentence
160,It's just a privilege to watch your mind at work.,SHELDON,[I never would have identified the fingerprint...,"[LEONARD, SHELDON]",BBT,True,4,4,"[4, 5]"
170,I don't think I'll be able to stop thinking ab...,PENNY,[This is one of my favorite places to kick bac...,"[HOWARD, PENNY, HOWARD, HOWARD, HOWARD, PENNY,...",BBT,True,4,4,"[4, 4, 4, 4, 4, 4, 4]"
180,"Since it's not bee season, you can have my epi...",SHELDON,"[Here we go. Pad thai, no peanuts., But does i...","[LEONARD, HOWARD, LEONARD]",BBT,False,4,4,"[4, 4, 4]"
190,"Lois Lane is falling, accelerating at an initi...",SHELDON,[A marathon? How many Superman movies are ther...,"[PENNY, SHELDON, PENNY, SHELDON, SHELDON, PENN...",BBT,False,4,4,"[4, 0, 4, 4, 4, 4, 4]"
1105,I'm just inferring this is a couch because the...,SHELDON,"[Great Caesar's ghost, look at this place., So...","[SHELDON, LEONARD, SHELDON, SHELDON, SHELDON, ...",BBT,True,4,2,"[2, 1, 1, 1, 0, 4]"
...,...,...,...,...,...,...,...,...,...
2169,"Hes not right for the part, and if I suggest h...",CHANDLER,"[What am I gonna do now?, Just pass the tape a...","[CHANDLER, RACHEL]",FRIENDS,True,0,0,"[0, 4]"
2235,"Oh yeah he has a caretaker his older brother, ...",CHANDLER,"[Helo! Anybody in there order a celebrity?, Wh...","[JOEY, PERSON, CHANDLER, PERSON]",FRIENDS,False,1,0,"[0, 4, 1, 4]"
234,Is it me or the greetings gone downhill around...,CHANDLER,"[Hey, You son of a bitch!]","[CHANDLER, JOEY]",FRIENDS,True,4,4,"[4, 0]"
2608,"You are right, by saying nice, I am virtually ...",CHANDLER,"[Did I go to this school?, Hey, there's Missy ...","[CHANDLER, ROSS, CHANDLER, ROSS]",FRIENDS,True,1,4,"[4, 4, 3, 1]"


In [30]:
# Define the keys for the floats
#keys = ['neg', 'neu', 'pos']

# Initialize a list to hold the resulting arrays
X_sentences = []
sentiment_per_context = df_h['sentiment_context_per_sentence']
# Process each array (list of dictionaries)
for idx, sentence in enumerate(sentiment_per_context):
    # Ensure each list has at least 3 dictionaries, padding if necessary
    sentence.extend([-1] * (3 - len(sentence)))# Classify as -1 if no context sentence
    sentence = sentence[:3]
    X_sentences.append(sentence)

print(X_sentences)

    # Extract the relevant values from the last 3 context sentences and the corresponding utterance
    #context_values = [value for sentence in sentences[-3:] for value in (sentence['neg'], sentence['neu'], sentence['pos'])]
    #utterance_values = [utterance[idx][key] for key in keys]

    # Combine the context and utterance values
    #X_sentences.append(context_values + utterance_values)

# Split the dataset into 80% training and 20% testing
#X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X_sentences, y, test_size=0.2, random_state=42)

# Create a DataFrame with appropriate column names
columns = ["Ctx_1", "Ctx_2", "Ctx_3"]
df_h_3 = pd.DataFrame(X_sentences, columns=columns)
#df_h_3 = pd.concat([df_h_3, df_h['sentiment_utterance']], axis=1)
df_h_3

[[4, 5, -1], [4, 4, 4], [4, 4, 4], [4, 0, 4], [2, 1, 1], [4, 4, 4], [3, 3, -1], [4, 4, 4], [5, 4, 1], [1, 0, 0], [5, 4, 4], [4, 4, 0], [4, 5, -1], [4, 5, 5], [1, 1, 4], [4, 0, 4], [0, 4, 4], [4, 4, 3], [3, 4, 4], [3, 4, 4], [4, -1, -1], [0, 0, 0], [4, 0, 0], [0, 4, 5], [0, 0, 4], [1, 4, 1], [5, 4, -1], [4, 4, 4], [4, 4, 4], [4, 0, 1], [4, 4, -1], [1, 0, -1], [4, 5, -1], [4, 0, 0], [4, 4, 3], [3, 4, 4], [4, 0, -1], [0, 4, -1], [4, 1, 4], [4, 4, 4], [0, 4, 4], [4, 0, -1], [4, 4, 4], [4, 4, 4], [4, 3, 3], [4, 0, 3], [4, 5, 4], [4, 4, 4], [0, 4, 4], [5, 1, 4], [4, 4, 4], [4, 4, 4], [4, 4, 0], [3, -1, -1], [4, 4, 4], [1, 2, 4], [3, 4, 4], [4, 5, 4], [4, 4, 0], [2, 4, 4], [4, 1, 4], [2, 4, 3], [4, 5, -1], [4, 0, 1], [4, 4, 4], [4, 4, 4], [0, 4, 4], [3, 4, 4], [4, 0, 0], [4, -1, -1], [0, 4, -1], [0, -1, -1], [4, 4, 4], [4, 4, 1], [4, 4, 1], [4, 4, 0], [3, 3, 3], [4, 4, 4], [3, 4, 4], [4, 3, 4], [4, 4, 3], [0, 0, 4], [3, 4, -1], [4, 4, -1], [3, 4, 1], [4, 4, 1], [4, 4, 4], [3, 4, 1], [4, 4, 5]

KeyError: 'sentiment_utterance'

In [ ]:
df_h

In [36]:
hartmann_sentiment(df_h['utterance'][160])

4

In [37]:
# Load the JSON file into a DataFrame
file_path = 'data/sarcasm_data.json'
df_h = pd.read_json(file_path).transpose()
# Apply sentiment analysis to the 'utterance' column
df_h['sentiment_utterance'] = df_h['utterance'].apply(hartmann_sentiment)

In [38]:
df_h

,utterance,speaker,context,context_speakers,show,sarcasm,sentiment_utterance
160,It's just a privilege to watch your mind at work.,SHELDON,[I never would have identified the fingerprint...,"[LEONARD, SHELDON]",BBT,True,4
170,I don't think I'll be able to stop thinking ab...,PENNY,[This is one of my favorite places to kick bac...,"[HOWARD, PENNY, HOWARD, HOWARD, HOWARD, PENNY,...",BBT,True,4
180,"Since it's not bee season, you can have my epi...",SHELDON,"[Here we go. Pad thai, no peanuts., But does i...","[LEONARD, HOWARD, LEONARD]",BBT,False,4
190,"Lois Lane is falling, accelerating at an initi...",SHELDON,[A marathon? How many Superman movies are ther...,"[PENNY, SHELDON, PENNY, SHELDON, SHELDON, PENN...",BBT,False,4
1105,I'm just inferring this is a couch because the...,SHELDON,"[Great Caesar's ghost, look at this place., So...","[SHELDON, LEONARD, SHELDON, SHELDON, SHELDON, ...",BBT,True,4
...,...,...,...,...,...,...,...
2169,"Hes not right for the part, and if I suggest h...",CHANDLER,"[What am I gonna do now?, Just pass the tape a...","[CHANDLER, RACHEL]",FRIENDS,True,0
2235,"Oh yeah he has a caretaker his older brother, ...",CHANDLER,"[Helo! Anybody in there order a celebrity?, Wh...","[JOEY, PERSON, CHANDLER, PERSON]",FRIENDS,False,1
234,Is it me or the greetings gone downhill around...,CHANDLER,"[Hey, You son of a bitch!]","[CHANDLER, JOEY]",FRIENDS,True,4
2608,"You are right, by saying nice, I am virtually ...",CHANDLER,"[Did I go to this school?, Hey, there's Missy ...","[CHANDLER, ROSS, CHANDLER, ROSS]",FRIENDS,True,1
